In [3]:
import requests
import pandas as pd
import io
from datetime import datetime, timedelta

# Constants
API_KEY = '2lZRFGaqFiEYkzr7WUuT4EaoC1X' #add the API
SINCE_DATE = int(datetime(2023, 1, 1).timestamp())  # Jan 1, 2023
UNTIL_DATE = int(datetime.now().timestamp())  # Current date

# URLs for fetching data
PRICE_URL = 'https://api.glassnode.com/v1/metrics/market/price_usd_close'
METRICS = [
    'https://api.glassnode.com/v1/metrics/derivatives/futures_volume_buy_perpetual_sum',
    'https://api.glassnode.com/v1/metrics/derivatives/futures_volume_sell_perpetual_sum'
    
]

def fetch_glassnode_data(url, asset='BTC'):
    params = {
        'a': asset,
        's': SINCE_DATE,
        'u': UNTIL_DATE,
        'api_key': API_KEY,
        'f': 'CSV',
        'c': 'USD'
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        df = pd.read_csv(io.StringIO(response.text))
        metric_name = url.split('/')[-1]
        df.columns = ['t', metric_name]
        df['t'] = pd.to_datetime(df['t'], unit='s')
        df[metric_name] = pd.to_numeric(df[metric_name], errors='coerce')
        return df
    else:
        print(f"Failed to fetch data from {url}. Status code: {response.status_code}")
        return None

# Fetch and merge data
price_df = fetch_glassnode_data(PRICE_URL)
all_dfs = [price_df]
for metric_url in METRICS:
    metric_df = fetch_glassnode_data(metric_url)
    if metric_df is not None:
        all_dfs.append(metric_df)

merged_df = pd.concat(all_dfs, axis=1)
merged_df = merged_df.loc[:,~merged_df.columns.duplicated()]
merged_df.set_index('t', inplace=True)
merged_df.tail()

,price_usd_close,futures_volume_buy_perpetual_sum,futures_volume_sell_perpetual_sum
t,,,
2024-10-02,60649.186895,2.347602e+10,2.321245e+10
2024-10-03,60740.512269,2.063772e+10,2.070962e+10
2024-10-04,62104.674280,1.708247e+10,1.694579e+10
2024-10-05,62075.343850,5.498388e+09,5.531406e+09
2024-10-06,62824.623690,7.566760e+09,7.361861e+09
